In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci

I build this script to specifically calcualte only part  of the electron electron interaction that coresponds to the seniority 0 part of the electron electron interaction from the 2RDM. To do so I believe I will have to spin resolve my caclulation somehow.

$$E_{ee} = \sum_p \sum_q \sum_r \sum_s \gamma_{2_{pqrs}} \langle p q | r s \rangle$$

In [2]:
def ee_from_gamma_2(eri,dm2):
    E = 0 
    dim = eri.shape[0]
    for p in range(0,dim):
        for q in range(0,dim):
            for r in range(0,dim):
                for s in range(0,dim):
                    if p == r and q == s:
                        E+=eri[p,q,r,s]*dm2[p,q,r,s]
                        print(f"[{p}{q}|{r}{s}]")
                    elif p == s and q == r:
                        E+=eri[p,q,r,s]*dm2[p,q,r,s]
                        print(f"[{p}{q}|{r}{s}]")
    
    return E

In [3]:
mol = gto.Mole()
mol.atom = """
    He    0.    0.    0.
"""
#mol.basis = "cc-pvdz"
mol.basis = "6-31g" 
mol.build()
print(mol.nelec)

(1, 1)


In [4]:
# Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()
h = mf.get_hcore()
J = mf.get_j()
K = mf.get_k()
C = mf.mo_coeff
h1 = C.T@h@C

converged SCF energy = -2.85516042615445


In [5]:
# Find electron-repulsion integrals (eri).
eri = ao2mo.kernel(mol, mf.mo_coeff)
eri = np.asarray(ao2mo.restore(1, eri, mol.nao))

In [6]:
print(eri.shape)

(2, 2, 2, 2)


In [7]:
# First create FCI solver with function fci.FCI and solve the FCI problem
#
cisolver = fci.FCI(mol, mf.mo_coeff)
e, fcivec = cisolver.kernel()

In [8]:
dm1, dm2 = cisolver.make_rdm12(fcivec, mf.mo_coeff.shape[1], mol.nelec)

In [13]:
print(.5*ee_from_gamma_2(eri,dm2), e-np.trace(np.matmul(dm1,h1)))


[00|00]
[01|01]
[01|10]
[10|01]
[10|10]
[11|11]
0.9959004643904699 0.9934201474038242
2.6610321481861328
